<h2> Lab 9

<h3> Exercise Set 1

In [2]:
import gymnasium as gym
import numpy as np
import tensorflow as tf #you can tf.keras to access most of the needed items
from collections import deque #get to know this package, it is amazing to build your replay buffer :)
import random

2024-10-28 20:12:49.751185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 20:12:49.903630: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 20:12:49.950122: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 20:12:50.208709: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-28 20:13:16.377695: W tensorflow/compiler/tf2

In [5]:
# Hyperparameters
learning_rate = 0.001 #use mse loss, adam optimizer, relu and linear activations
gamma = 0.99 #reward discount factor
epsilon_init = 1.0 #start with full exploration
epsilon_min = 0.01 #towards the end you are only 10\% likely to explore, otherwise exploit
epsilon_decay = 0.995 #decay after every episode
batch_size = 64 #for gradient descent
memory_size = 2000 #size of the replay buffer
target_update_freq = 10 #parameter C in slides, when to copy the primary to the target network
num_episodes = 500 #total episodes for training
num_layers = [64,64] #these are hidden layers only for the Q/primary network
num_epochs = 3 #only fit the primary/Q network for 3 epochs to finish fast

# Create the environment (for testing )
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0] #this gives you indication of the state space
action_size = env.action_space.n #this gives you indication of the action space

In [17]:
# 1. Initialize replay memory (ReplayBuffer) to store past experiences
replayBuffer = deque()

# 2. Initialize Q−network (a.k.a Primary Network) with random weights
primaryQN = tf.keras.Sequential()
primaryQN.add(tf.keras.Input(shape=(4,)))
primaryQN.add(tf.keras.layers.Dense(50, activation='relu'))
primaryQN.add(tf.keras.layers.Dense(50, activation='relu'))
primaryQN.add(tf.keras.layers.Dense(2, activation='relu'))

# 3. Initialize target Q−network (Target−Q−network) with same weights as Q−network
targetQN = tf.keras.Sequential()
targetQN.add(tf.keras.Input(shape=(4,)))
targetQN.add(tf.keras.layers.Dense(50, activation='relu'))
targetQN.add(tf.keras.layers.Dense(50, activation='relu'))
targetQN.add(tf.keras.layers.Dense(2, activation='relu'))
targetQN.set_weights(primaryQN.get_weights())

In [ ]:
epsilon = epsilon_init

for i in range(num_episodes):
    state = env.reset()
    done = False
    for j in range(500):
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample()
        else:
            action = targetQN.predict(state)

        nextState, reward, done, info = env.step(action)
        replayBuffer.append([state, action, reward, nextState])

        # 14. If replay memory contains enough samples:
        if len(replayBuffer) >= batch_size:
            # 15. Sample a batch of transitions (state , action , reward, next state) from replay memory

            # 16. For each transition in the mini−batch: #this is the most tricky part!
                # 17. If next state is terminal:
                    # 18. Set target = reward
                # 19. Otherwise:
                    # 20. Set target = reward + discount factor ∗ max(Target−Q−network(next state))
                # 21. Perform gradient descent step to minimize loss :
                    # loss = (target − Q−network(state, action))ˆ2
                #22. Update Q−network weights using gradient descent
                
        state = nextState

        if j % target_update_freq == 0:
              targetQN.set_weights(primaryQN.get_weights())

        epsilon = epsilon * epsilon_decay

